In [8]:
import torch
import torch.nn as nn

# 定义输入张量，形状为 (batch_size, channels, height, width)
input_tensor = torch.randn(4, 3, 32, 32)  # 4个样本，3个通道，32x32的图像

# 创建一个卷积层
conv_layer_1 = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3)  # 输出通道数为4
conv_layer_2 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)  # 输出通道数为8

# 前向传播
output_tensor_1 = conv_layer_1(input_tensor)
output_tensor_2 = conv_layer_2(input_tensor)
print(conv_layer_1.weight.shape)
print("输出张量形状 (4 通道):", output_tensor_1.shape)
print(conv_layer_2.weight.shape)
print("输出张量形状 (8 通道):", output_tensor_2.shape)

torch.Size([4, 3, 3, 3])
输出张量形状 (4 通道): torch.Size([4, 4, 30, 30])
torch.Size([8, 3, 3, 3])
输出张量形状 (8 通道): torch.Size([4, 8, 30, 30])
